In [1]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse as spa
import scipy.sparse.linalg as spalin
from scipy.optimize import fsolve
from tqdm import trange
import matplotlib.colors as mcolors
from matplotlib.ticker import MaxNLocator
import os
import sys
sys.path.append("../dyn/")
sys.path.append("../")
sys.path.append("../phase/")
sys.path.append("../artfigs_NC/")
from dyn_ultis import *
from spatial_ultis import *
from artfigs_NC_params import *
from artfigs_NC_ultis import *

In [17]:

def find_dyn_fix_point_with_variance(p_net: Network_Params, p_simul: Simul_Params, dim=2):
    if type(p_simul.activation_func) == str:
        activation_func_list = [activation_func_dict[p_simul.activation_func], activation_func_dict[p_simul.activation_func]]
    elif type(p_simul.activation_func) == list:
        activation_func_list = [activation_func_dict[p_simul.activation_func[0]], activation_func_dict[p_simul.activation_func[1]]]
    external_input = external_input_dict[p_simul.external_input]        
        
    #x定义为前两个是E跟I的均值，而后两个则是方差
    def df(x):
        dx_dt = np.array([0.0,0.0,0.0,0.0])
        dx_dt[0] = -x[0] + p_net.g_bar_EE * gauss_int(x[0], x[2], activation_func_list[0]) + p_net.g_bar_EI * p_net.N_I / p_net.N_E * gauss_int(x[1], x[3], activation_func_list[1]) + np.mean(external_input(0, p_net)[0][0:p_net.N_E])
        dx_dt[1] = -x[1] + p_net.g_bar_IE * gauss_int(x[0], x[2], activation_func_list[0]) * p_net.N_E / p_net.N_I + p_net.g_bar_II * gauss_int(x[1], x[3], activation_func_list[1]) + np.mean(external_input(0, p_net)[0][p_net.N_E:p_net.N_E+p_net.N_I])
        
        J_EE, J_EI, J_IE, J_II = p_net.g_bar_EE/p_net.N_EE, p_net.g_bar_EI/p_net.N_EI, p_net.g_bar_IE/p_net.N_IE, p_net.g_bar_II/p_net.N_II
        sigma_EE, sigma_EI, sigma_IE, sigma_II = p_net.g_EE/np.sqrt(p_net.N_EE), p_net.g_EI/np.sqrt(p_net.N_EI), p_net.g_IE/np.sqrt(p_net.N_IE), p_net.g_II/np.sqrt(p_net.N_II)  
        if dim == 1:
            sigma_eff_EE = p_net.N_EE * (p_net.N_E/p_net.N_E) * ((1 - p_net.N_EE/(2 * np.sqrt(np.pi)*p_net.d_EE*(p_net.N_E))) * J_EE **2 + sigma_EE**2)
            sigma_eff_IE = p_net.N_IE * (p_net.N_E/p_net.N_I) * ((1 - p_net.N_IE/(2 * np.sqrt(np.pi)*p_net.d_IE*(p_net.N_I))) * J_IE **2 + sigma_IE**2)
            sigma_eff_EI = p_net.N_EI * (p_net.N_I/p_net.N_E) * ((1 - p_net.N_EI/(2 * np.sqrt(np.pi)*p_net.d_EI*(p_net.N_E))) * J_EI **2 + sigma_EI**2)
            sigma_eff_II = p_net.N_II * (p_net.N_I/p_net.N_I) * ((1 - p_net.N_II/(2 * np.sqrt(np.pi)*p_net.d_II*(p_net.N_I))) * J_II **2 + sigma_II**2)
        elif dim == 2:
            sigma_eff_EE = p_net.N_EE * (p_net.N_E/p_net.N_E) * ((1 - p_net.N_EE/(4 * np.pi * p_net.d_EE**2 * p_net.N_E)) * J_EE **2 + sigma_EE**2)
            sigma_eff_IE = p_net.N_IE * (p_net.N_E/p_net.N_I) * ((1 - p_net.N_IE/(4 * np.pi * p_net.d_IE**2 * p_net.N_I)) * J_IE **2 + sigma_IE**2)
            sigma_eff_EI = p_net.N_EI * (p_net.N_I/p_net.N_E) * ((1 - p_net.N_EI/(4 * np.pi * p_net.d_EI**2 * p_net.N_E)) * J_EI **2 + sigma_EI**2)
            sigma_eff_II = p_net.N_II * (p_net.N_I/p_net.N_I) * ((1 - p_net.N_II/(4 * np.pi * p_net.d_II**2 * p_net.N_I)) * J_II **2 + sigma_II**2)
        else:
            print('dimenstion error')
            return None       
        
        #TODO 这边存噪声强度没得给出
        dx_dt[2] = -x[2] + sigma_eff_EE * gauss_int(x[0], x[2], lambda x:activation_func_list[0](x)**2) + sigma_eff_EI * gauss_int(x[1], x[3], lambda x:activation_func_list[1](x)**2) + 0.01
        dx_dt[3] = -x[3] + sigma_eff_IE * gauss_int(x[0], x[2], lambda x:activation_func_list[0](x)**2) + sigma_eff_II * gauss_int(x[1], x[3], lambda x:activation_func_list[1](x)**2) + 0.01
        return dx_dt
    
    # initial_guesses = [np.random.uniform(low=0, high=5, size=(4,)) for _ in range(50)]
    initial_guesses = [np.random.uniform(low=0, high=10, size=(4,)) for _ in range(guess_num)] #TEMP

    fixed_points = []
    for guess in initial_guesses:
        point = fsolve(df, guess)
        if np.sqrt((np.array(df(point)) ** 2).sum()) < 1e-3:
            if not any(np.allclose(point, fp, atol=1e-3) for fp in fixed_points):
                fixed_points.append(point)
    return fixed_points



In [2]:
p_simul = Simul_Params(T = 2000, t_step=5, record_step=10, activation_func=['thres_linear','thres_powerlaw'], external_input="DC_noise",tau_m=20.0)
p_net = generate_params_phase_d_II_g_bar_II_thres_L(15,5)

In [4]:
p_simul = Simul_Params(T = 2000, t_step=5, record_step=10, activation_func=['tanh','tanh_high'], external_input="noise",tau_m=20.0)
p_net = generate_params_phase_g_d_II_L_chaos(15, 15)

In [5]:
find_dyn_fix_point_with_variance_tanh(p_net, p_simul)

[[0.0, 0.0, 0.9691331179195454, 3.92095837238724]]